In [1]:
!pip install PrettyTable
!pip install chainer

scikit-umfpack 0.3.2 has requirement numpy>=1.15.3, but you'll have numpy 1.15.1 which is incompatible.
menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 2.2.3 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.2.0 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.1.0 which is incompatible.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
scikit-umfpack 0.3.2 has requirement numpy>=1.15.3, but you'll have numpy 1.15.1 which is incompatible.
menpo 0.8.1 has requirement matplotlib<2.0,>=1.4, but you'll have matplotlib 2.2.3 which is incompatible.
menpo 0.8.1 has requirement pillow<5.0,>=3.0, but you'll have pillow 5.2.0 which is incompatible.
menpo 0.8.1 has requirement scipy<1.0,>=0.16, but you'll have scipy 1.1.0 which is incompatible.
You are using pip version 10.0.1, however ve

In [2]:
from prettytable import PrettyTable
from itertools import compress 
import numpy as np
from matplotlib import pyplot as plt
from numpy import linalg
import time
import operator
import chainer
import pandas as pd
chainer.print_runtime_info()
full_all_digits = np.loadtxt('zipcombo.dat.txt')

Platform: Linux-4.15.0-1032-aws-x86_64-with-debian-stretch-sid
Chainer: 7.0.0
ChainerX: Not Available
NumPy: 1.15.1
CuPy: Not Available
iDeep: Not Available


In [3]:
# Assign x and y 
full_digits_x = full_all_digits[:,1:]
full_digits_y = full_all_digits[:, 0]

# Placeholders for randomize()
random_split = 0
full_digits_x = 0
full_digits_y = 0
random_split_x = 0
random_split_x = 0
train_data_x = 0
test_data_x = 0
train_data_y = 0
test_data_y = 0
random_split_train = 0
random_split_test = 0
runs = []

def randomize():
    """
    Shuffle the data set to separate train and test set
    """
    global random_split 
    global full_digits_x 
    global full_digits_y 
    global random_split_x 
    global random_split_y 
    global test_data_x
    global train_data_x
    global train_data_y
    global test_data_y
    global random_split_train
    global random_split_test

    random_split = full_all_digits.copy()
    np.random.shuffle(random_split)
    
    full_digits_x = random_split[:,1:]
    full_digits_y = random_split[:, 0]
    
    random_split_train = random_split[:7438, :]
    random_split_test = random_split[7438:,:]
    
    random_split_x = full_digits_x.copy()
    random_split_y = full_digits_y.copy()

    train_data_x = random_split_train[:,1:]
    test_data_x = random_split_test[:,1:]
    
    train_data_y = random_split_train[:,0]
    test_data_y = random_split_test[:,0]
    
randomize()

In [4]:
test_fold_1 = 0
test_fold_2 = 0
test_fold_3 = 0
test_fold_4 = 0
test_fold_5 = 0

training_fold_1 = 0
training_fold_2 = 0
training_fold_3 = 0
training_fold_4 = 0
training_fold_5 = 0

training_fold_x = 0
training_fold_y = 0

test_fold_x = 0
test_fold_y = 0
def randomize2():
    """
    Making a framework for 5-fold cross validation
    """
    global test_fold_1
    global test_fold_2
    global test_fold_3
    global test_fold_4
    global test_fold_5

    global training_fold_1
    global training_fold_2
    global training_fold_3
    global training_fold_4
    global training_fold_5

    global training_fold_x
    global training_fold_y

    global test_fold_x
    global test_fold_y
    

    np.random.shuffle(random_split_train)
    
    train_data = random_split_train
    
    test_fold_1 = train_data[0:1487,]
    test_fold_2 = train_data[1487:2975,]
    test_fold_3 = train_data[2975:4463,]
    test_fold_4 = train_data[4463:5951,]
    test_fold_5 = train_data[5951:7439,]

    training_fold_1 = np.delete(train_data, range(0,1487), 0)
    training_fold_2 = np.delete(train_data, range(1487,2975), 0)
    training_fold_3 = np.delete(train_data, range(2975,4463), 0)
    training_fold_4 = np.delete(train_data, range(4463,5951), 0)
    training_fold_5 = np.delete(train_data, range(5951,7439), 0)

    training_fold_x = [training_fold_1[:,1:], training_fold_2[:,1:], training_fold_3[:,1:], training_fold_4[:,1:], training_fold_5[:,1:]]
    training_fold_y = [training_fold_1[:,0].reshape(1,training_fold_1[:,0].size), training_fold_2[:,0].reshape(1,training_fold_2[:,0].size), training_fold_3[:,0].reshape(1,training_fold_3[:,0].size), training_fold_4[:,0].reshape(1,training_fold_4[:,0].size), training_fold_5[:,0].reshape(1,training_fold_5[:,0].size)]

    test_fold_x = [test_fold_1[:,1:], test_fold_2[:,1:], test_fold_3[:,1:], test_fold_4[:,1:], test_fold_5[:,1:]]
    test_fold_y = [test_fold_1[:,0].reshape(1,test_fold_1[:,0].size), test_fold_2[:,0].reshape(1,test_fold_2[:,0].size), test_fold_3[:,0].reshape(1,test_fold_3[:,0].size), test_fold_4[:,0].reshape(1,test_fold_4[:,0].size), test_fold_5[:,0].reshape(1,test_fold_5[:,0].size)]

randomize2()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:55: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.


In [5]:
# Add columns to train_data_x
testlist = train_data_x[1]
numlist = np.arange(0,len(testlist))

colname = ['Column'+str(i) for i in numlist]
train_df = pd.DataFrame(np.vstack(train_data_x),columns = colname)
test_df = pd.DataFrame(np.vstack(test_data_x),columns = colname)

In [6]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

# n_estimators = The number of trees in the forest
# Default: (n_estimators, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
# Create Random Forest Classifier
rf = RandomForestClassifier(n_estimators = 100)

# Meta-transformer for selecting features based on importance weights.
# Default: (estimator, threshold=None, prefit=False, norm_order=1, max_features=None)
# SelectFromModel will select those features which importance is greater than the mean importance of all the features by default
sel = SelectFromModel(rf)

# To see which features are important we can use get_support method on the fitted model.
# True for the features whose importance is greater than the mean importance and False for the rest.
sel.fit(train_df, train_data_y)
sel.get_support()

# We can now make a list for the selected features.
selected_feat= train_df.columns[(sel.get_support())]


/usr/local/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # in our dataset format, the label is always the last column
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [8]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for Color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question. See the demo below.
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        return val >= self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            colname[self.column], condition, str(self.value))

In [9]:
def partition(rows, question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [10]:
def gini(rows):
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [11]:
def info_gain(left, right, current_uncertainty):
    """Information Gain.

    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [12]:
def find_best_split(rows,selected_features):
    """Find the best question to ask by iterating over selected feature / value
    and calculating the information gain."""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    #n_features = len(rows[0]) - 1  # number of columns

    for feature in selected_features:  # for each feature
        col = colname.index(feature)
        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [13]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Apple") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [14]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [15]:
def build_tree(rows,selected_features):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows,selected_features)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows,selected_features)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows,selected_features)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [16]:
def print_tree(node, spacing=""):
    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [17]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [18]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [19]:
def buildforest(x, y, testx, testy, imp_features, n_features, n_trees):
    tree_list=[]
    oob_list=[]

    # Change x to df
    train_df_og = pd.DataFrame(np.vstack(x),columns = colname)
    # add label to the dataframe
    train_df_og["label"] = y

    # ========= BUILD TREE =========
    for i in range(n_trees):
        # Select 10 random features
        headers = np.random.permutation(imp_features)[:n_features]
        #print("modeling with ",headers)
        if i%3==0:
            print("building ",i+1,"th tree with ",headers)
        # Sampling with replacement
        training_data_temp = train_df_og.sample(n=len(train_df_og), replace=True)
        
        #print("data use to build a tree: ", np.shape(training_data_temp))
        
        # OOB samples
        oob_samples= pd.merge(train_df_og, training_data_temp, how='left', indicator=True) \
            .query("_merge == 'left_only'") \
            .drop('_merge',1)

        oob_samples_idx = list(oob_samples.index.values.tolist())

        # Change to lists
        training_data = training_data_temp.values.tolist()

        # Build a decision tree
        my_tree = build_tree(training_data,headers)
        # print_tree(my_tree)
        tree_list.append(my_tree)

        # OOB append
        oob_list.append(oob_samples_idx)

    # ========= CALCULATE OOB ERROR FOR TRAIN DATA =========
    training_data_eval = train_df_og.values.tolist()

    trainingscore=0
    boollist=None
    for i in range(len(training_data_eval)):
        # print("index: ",i)

        # Search for trees that are not built on this value
        boollist = [i not in sublist for sublist in oob_list]
        evaltree = list(compress(tree_list, boollist))
        
        label = training_data_eval[i][-1]
        predictions =[]
        if len(evaltree)!=0:
            for tree in evaltree:
                a = print_leaf(classify(training_data_eval[i], tree))
                pred = max(a.items(), key=operator.itemgetter(1))[0]
                predictions.append(pred)
                
        
            # print("label: ",label)
            # print(predictions)

            pred_of_trees = max(predictions, key=predictions.count)
            if label == pred_of_trees:
                trainingscore+=1
        
    # ========= CALCULATE TEST ERROR =========
    test_df_og = pd.DataFrame(np.vstack(testx),columns = colname)
    test_df_og['label'] = testy
    test_data_eval = test_df_og.values.tolist()

    testscore = 0
    for i in range(len(test_data_eval)): #for one line of data
        label = test_data_eval[i][-1]
        testpredictions=[]
        for tree in tree_list:
            a = print_leaf(classify(test_data_eval[i], tree))
            pred = max(a.items(), key=operator.itemgetter(1))[0]
            testpredictions.append(pred)
        
        pred_of_trees = max(testpredictions, key=testpredictions.count)
        if label == pred_of_trees:
            testscore+=1
            
    print("trainacc: ", trainingscore/len(training_data_eval))
    print("testacc: ",testscore/len(test_data_eval))

    return trainingscore/len(training_data_eval), testscore/len(test_data_eval)

In [20]:
d_list = [7,8,9,10,11,12,13]
train_test_store_RF = np.zeros((7,2,5))

for i in range(5): #20 run
    randomize()
    indx = 0
    for d in d_list:
        print ("# of features:", d,"||", "# of run =",i)
        start = time.time()
        result = buildforest(train_data_x, train_data_y, test_data_x, test_data_y, selected_feat, d, 10)
        train_test_store_RF[indx,0,i] = result[0]
        train_test_store_RF[indx,1,i] = result[1]
        indx = indx+1
        end = time.time()
        print('%0.2f minutes' %((end - start) / 60))


# of features: 7 || # of run = 0
building  1 th tree with  ['Column59' 'Column75' 'Column25' 'Column91' 'Column101' 'Column68'
 'Column139']
building  4 th tree with  ['Column105' 'Column229' 'Column106' 'Column89' 'Column182' 'Column36'
 'Column53']
building  7 th tree with  ['Column108' 'Column121' 'Column102' 'Column122' 'Column55' 'Column87'
 'Column137']
building  10 th tree with  ['Column84' 'Column92' 'Column117' 'Column68' 'Column162' 'Column182'
 'Column211']
trainacc:  0.9912610916913148
testacc:  0.8279569892473119
27.30 minutes
# of features: 8 || # of run = 0
building  1 th tree with  ['Column164' 'Column146' 'Column83' 'Column103' 'Column87' 'Column180'
 'Column181' 'Column52']
building  4 th tree with  ['Column115' 'Column100' 'Column106' 'Column74' 'Column169' 'Column102'
 'Column153' 'Column52']
building  7 th tree with  ['Column182' 'Column130' 'Column59' 'Column117' 'Column204' 'Column199'
 'Column188' 'Column103']
building  10 th tree with  ['Column119' 'Column165' 

,d=7,d=8,d=9,d=10,d=11,d=12,d=13
Train Error,0.99239±0.00112,0.99301±0.00094,0.99363±0.00073,0.99449±0.00077,0.99532±0.00049,0.99503±0.00073,0.99524±0.00050
Test Error,0.82946±0.00449,0.85237±0.00877,0.87430±0.00351,0.87742±0.00617,0.89333±0.00624,0.89710±0.00580,0.90194±0.00303


In [21]:

# Create table
train_test_store_RF = 1 - train_test_store_RF
meanval = train_test_store_RF.mean(axis=2) #(7,20)
stdval = train_test_store_RF.std(axis=2)

colval = ['d=7','d=8','d=9','d=10','d=11','d=12','d=13']
df = pd.DataFrame(index=['Train Error','Test Error'], columns=colval)

for i in range(7):
    df.at['Train Error', colval[i]] = str('%.5f'%(meanval[i,0])) +'±' + str('%.5f'%(stdval[i,0]))
    df.at['Test Error', colval[i]] = str('%.5f'%(meanval[i,1])) + '±' + str('%.5f'%(stdval[i,1]))

# display table
df




,d=7,d=8,d=9,d=10,d=11,d=12,d=13
Train Error,0.00761±0.00112,0.00699±0.00094,0.00637±0.00073,0.00551±0.00077,0.00468±0.00049,0.00497±0.00073,0.00476±0.00050
Test Error,0.17054±0.00449,0.14763±0.00877,0.12570±0.00351,0.12258±0.00617,0.10667±0.00624,0.10290±0.00580,0.09806±0.00303
